In [1]:

pip install googlemaps


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached requests-2.32.3-py3-none-any.whl (64 kB)
  Using cached charset_normalizer-3.3.2-cp311-cp311-win_amd64.whl (99 kB)
  Using cached urllib3-2.2.2-py3-none-any.whl (121 kB)
  Using cached certifi-2024.7.4-py3-none-any.whl (162 kB)
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'c:\\Python311\\Scripts\\normalizer.exe' -> 'c:\\Python311\\Scripts\\normalizer.exe.deleteme'


[notice] A new release of pip available: 22.3 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install pandas openpyxl googlemaps


  Using cached googlemaps-4.10.0.tar.gz (33 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached requests-2.32.3-py3-none-any.whl (64 kB)
  Using cached certifi-2024.7.4-py3-none-any.whl (162 kB)
  Running setup.py install for googlemaps: started
  Running setup.py install for googlemaps: finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  DEPRECATION: googlemaps is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559

[notice] A new release of pip available: 22.3 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install pandas openpyxl googlemaps xlsxwriter


     ------------------------------------ 159.9/159.9 kB 870.3 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [Errno 13] Permission denied: 'c:\\Python311\\Scripts\\vba_extract.py'
Consider using the `--user` option or check the permissions.


[notice] A new release of pip available: 22.3 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
pip install pandas openpyxl googlemaps


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# Importing necessary libraries
import pandas as pd                  # Library for data manipulation and analysis
import googlemaps                    # Google Maps API client for Python
import xlsxwriter                    # Library for writing Excel files
from googlemaps.exceptions import ApiError  # Importing exception handling from Google Maps API client

# Replace 'YOUR_API_KEY' with your actual API key
API_KEY = 'XXXXXXXXXXXXXXX YOUR GOOGLE MAP API KEY'  # Assigning my Google Maps API key to the variable API_KEY
gmaps = googlemaps.Client(key=API_KEY)               # Initializing the Google Maps client with the API key

# Read the Excel file containing the list of localities and types
file_path = 'localities.xlsx'        # Path to the Excel file
try:
    df = pd.read_excel(file_path)    # Reading the Excel file into a pandas DataFrame
except FileNotFoundError as e:       # Handling file not found error
    print(e)
    print(f"Please make sure that the Excel file '{file_path}' exists.")
    raise
except ValueError as e:              # Handling value error (e.g., wrong format or missing columns)
    print(e)
    print("Please make sure that the Excel file contains the required columns 'Sheet1' and 'Types'.")
    raise

# Function to get latitude and longitude for a given locality
def get_lat_long(locality):
    try:
        geocode_result = gmaps.geocode(locality)  # Geocoding the locality to get latitude and longitude
        if geocode_result:
            location = geocode_result[0]['geometry']['location']  # Extracting location from geocode result
            return (location['lat'], location['lng'])            # Returning latitude and longitude
        return (None, None)
    except ApiError as e:  # Handling API errors
        print(f"Geocode API error for locality '{locality}': {e}")
        return (None, None)

# Function to search for places and handle pagination
def search_places(lat, lng, place_type):
    all_results = []  # List to store all results
    page_token = None
    
    while True:
        try:
            places_result = gmaps.places_nearby(location=(lat, lng), radius=5000, type=place_type, page_token=page_token)  # Searching for places nearby
            all_results.extend(places_result['results'])  # Adding results to the list
            page_token = places_result.get('next_page_token')  # Getting the next page token for pagination
            if not page_token:  # Breaking the loop if no more pages
                break
        except ApiError as e:  # Handling API errors
            print(f"Places API error for location ({lat, lng}) and type '{place_type}': {e}")
            break
    
    return all_results  # Returning all results

# Function to get details of a place including contact number
def get_place_details(place_id):
    try:
        place_details = gmaps.place(place_id=place_id)  # Getting place details using place ID
        result = place_details.get('result', {})  # Extracting result from place details
        return {
            'Name': result.get('name', 'N/A'),
            'Address': result.get('vicinity', 'N/A'),
            'Rating': result.get('rating', 'N/A'),
            'Total Ratings': result.get('user_ratings_total', 'N/A'),
            'Contact Number': result.get('formatted_phone_number', 'N/A')
        }  # Returning a dictionary with place details
    except ApiError as e:  # Handling API errors
        print(f"Places Details API error for place ID '{place_id}': {e}")
        return {
            'Name': 'N/A',
            'Address': 'N/A',
            'Rating': 'N/A',
            'Total Ratings': 'N/A',
            'Contact Number': 'N/A'
        }  # Returning a dictionary with default values in case of error

# Initialize an empty list to store all the results
all_places = []

# Loop through each row and search for each locality and type
for index, row in df.iterrows():  # Iterating through each row in the DataFrame
    locality = row['Sheet1']  # Getting locality from the 'Sheet1' column
    place_type = row['Types']  # Getting place type from the 'Types' column
    lat, lng = get_lat_long(locality)  # Getting latitude and longitude for the locality
    
    if lat and lng:  # If valid latitude and longitude are found
        print(f"Searching for {place_type} near {locality} ({lat}, {lng})")
        places_results = search_places(lat, lng, place_type)  # Searching for places
        
        for place in places_results:  # Looping through the places found
            place_data = get_place_details(place['place_id'])  # Getting details for each place
            place_data['Locality'] = locality  # Adding locality to the place data
            place_data['Type'] = place_type  # Adding place type to the place data
            all_places.append(place_data)  # Adding place data to the list
            
            # Save results to an Excel file after every 50 entries
            if len(all_places) % 50 == 0:
                df_results = pd.DataFrame(all_places)  # Creating a DataFrame from the list of places
                df_results.to_excel(f'places_results_{len(all_places)//50}.xlsx', index=False)  # Saving the DataFrame to an Excel file
                print(f"Saved {len(all_places)} entries to places_results_{len(all_places)//50}.xlsx")
                
    else:  # If latitude and longitude are not found
        print(f"Could not find geocode for {locality}")

# Save any remaining results to an Excel file
if all_places:  # If there are any remaining places in the list
    df_results = pd.DataFrame(all_places)  # Creating a DataFrame from the list of places
    df_results.to_excel(f'places_results_{(len(all_places)-1)//50 + 1}.xlsx', index=False)  # Saving the DataFrame to an Excel file
    print(f"Saved final results to places_results_{(len(all_places)-1)//50 + 1}.xlsx")


Searching for hotel near Chandmeri, Varanasi (25.382963, 82.97199770000002)
Places API error for location (25.382963, 82.97199770000002) and type 'hotel': INVALID_REQUEST
Searching for restaurant near Dafi, Varanasi (25.2444365, 82.9797246)
Places API error for location (25.2444365, 82.9797246) and type 'restaurant': INVALID_REQUEST
Searching for school near Sarnath, Varanasi (25.3715564, 83.0252136)
Places API error for location (25.3715564, 83.0252136) and type 'school': INVALID_REQUEST
Saved 50 entries to places_results_1.xlsx
Searching for nan near Suswahi, Varanasi (25.2559729, 82.9743495)
Places API error for location (25.2559729, 82.9743495) and type 'nan': INVALID_REQUEST
Searching for nan near Lahurabir, Varanasi (25.3217454, 83.0015568)
Places API error for location (25.3217454, 83.0015568) and type 'nan': INVALID_REQUEST
Saved 100 entries to places_results_2.xlsx
Searching for nan near Ramnagar, Varanasi (25.27646, 83.0334524)
Places API error for location (25.27646, 83.0334

KeyboardInterrupt: 